In [11]:
import pandas as pd
import random
import time
import pymongo
from pymongo import MongoClient
from parsel import Selector
from datetime import datetime
from time import sleep
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import itertools

# Connecting ot db

connection = MongoClient("mongodb://localhost:27017")
db = connection["IndeedJobPostDB"]

chrome_options = Options()

PROXY = "" # HOST:PORT

chrome_options = uc.ChromeOptions()
chrome_options.add_argument(f'--proxy-server={PROXY}')

driver = uc.Chrome(executable_path=r'chromedriver', options=chrome_options)

position = input('Enter your preferred skill: ').strip()
place = input('Enter the location: ').strip()
no_of_pages = int(input('Enter the number of pages to scrape: '))

Enter your preferred skill: Restaurant
Enter the location: New York, NY
Enter the number of pages to scrape: 5


In [13]:
job_data = []
for page in range(0, no_of_pages*10, 10):
    driver.get(f'https://www.indeed.com/jobs?q={position}&l={place}&sort=date&start={page}')
    time.sleep(random.uniform(8.5, 10.9))

    try:
        close = driver.find_element(By.XPATH, '//div[@class="icl-CloseButton icl-Modal-close"]')
        close.click()
    except:
        pass

    soup = BeautifulSoup(driver.page_source, features="html.parser")
    divs = soup.find_all('div', 'slider_item css-kyg8or eu4oa1w0')
    print(len(divs))
    for item in divs:

        # Getting the Job Title
        title = item.find('a').text.strip()
        #print(title)
        type_of_job = ''
        salary = ''
        schedule = ''
        # Getting the job type
        try:
            all_metadata = item.find('div', 'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly')
            print(all_metadata.text)
            rows =[i.get_text(strip=True) for i  in all_metadata.find_all('div', 'metadata') if i.get_text(strip=True)!=""]
            print(rows)
            for element in rows:
                if '$' in element:
                    salary = element.strip()
                    rows.remove(element)
                    print(salary)
                else:
                    salary = ''
            for element in rows:
                if 'time' in element:
                    if '+1' in element:
                        type_of_job = 'Full-time, Part-time'
                    else:
                        type_of_job = element
                    rows.remove(element)
                    print(type_of_job)
                else:
                    type_of_job = ''
            if len(rows) > 0:
                schedule = rows[0]
                print(schedule)
            else:
                schedule = ''
        except:
            try:
                salary = item.find('div', 'attribute_snippet').text.strip()
                type_of_job = ''
            except:
                salary = ''

        try:
            company = item.find('span', 'companyName').text.strip()
        except:
            company = ''

        #Getting the Location
        try:
            location = item.find('div', 'companyLocation').text.strip()
        except:
            location = ''

        #Getting the link
        try:
            link = 'https://www.indeed.com/' + item.h2.a.get('href')
        except:
            link = ''

        # Date Posted
        try:
            date_posted = item.find('span', 'date').text.strip()
        except:
            date_posted = ''

        # Current Date
        try: 
            current_date = datetime.today().strftime('%y-%m-%d')
        except:
            current_date = ''

        try:
            description  = item.find('div', 'job-snippet').text.strip().replace('\n', ' ')
        except:
            description = ''

        data = {
            'Title' : title,
            'Job_Type' : type_of_job,
            'Company' : company,
            'Salary' : salary,
            'Schedule': schedule,
            'Location' : location,
            'Date_Posted' : date_posted.removeprefix('Posted'),
            'Current_Date' : current_date,
            'Summary' : description,
            'Apply_link' : link
        }
        job_data.append(data)
        print('[*] Saving')


df = pd.DataFrame(job_data)

15
$17.50 an hourFull-timeMonday to Friday
['$17.50 an hour', 'Full-time', 'Monday to Friday']
$17.50 an hour
Full-time
Monday to Friday
[*] Saving
$22 an hourFull-timeMonday to Friday
['$22 an hour', 'Full-time', 'Monday to Friday']
$22 an hour
Full-time
Monday to Friday
[*] Saving
Estimated $29.3K - $37.2K a yearFull-timeWeekend availability +2
['Estimated $29.3K - $37.2K a year', 'Full-time', 'Weekend availability+2']
Estimated $29.3K - $37.2K a year
Full-time
Weekend availability+2
[*] Saving
Estimated $23.5K - $29.8K a year
['Estimated $23.5K - $29.8K a year']
Estimated $23.5K - $29.8K a year
[*] Saving
$70,000 - $85,000 a yearFull-timeWeekend availability +2
['$70,000 - $85,000 a year', 'Full-time', 'Weekend availability+2']
$70,000 - $85,000 a year
Full-time
Weekend availability+2
[*] Saving
$85,000 - $95,000 a yearFull-time
['$85,000 - $95,000 a year', 'Full-time']
$85,000 - $95,000 a year
Full-time
[*] Saving
Full-time +1Weekend availability +1
['Full-time+1', 'Weekend availab

In [15]:
df_NY = pd.DataFrame(job_data)

In [17]:
df

,Title,Job_Type,Company,Salary,Schedule,Location,Date_Posted,Current_Date,Summary,Apply_link
0,DISHWASHER (FULL-TIME),Full-time,Restaurant Associates,,Monday to Friday,"New York, NY 10011 (Chelsea area)",Just posted,23-06-08,"Summary: Maintains dishes, pots, pans, trays, ...",https://www.indeed.com//rc/clk?jk=b86ec462493f...
1,GRILL COOK (FULL-TIME),Full-time,Restaurant Associates,,Monday to Friday,"New York, NY 10019 (Midtown area)",Just posted,23-06-08,"Serves customers in an efficient, friendly man...",https://www.indeed.com//rc/clk?jk=3fb1d1417712...
2,Server,Full-time,Marriott Vacations Worldwide,,Weekend availability+2,"New York, NY 10018 (Garment District area)",Just posted,23-06-08,Welcome and acknowledge all guests according t...,https://www.indeed.com//rc/clk?jk=fa3d00337861...
3,Server,,Dos Caminos - Meatpacking,Estimated $23.5K - $29.8K a year,,"New York, NY 10014 (West Village area)",Just posted,23-06-08,"Dental, vision, voluntary life, short term dis...",https://www.indeed.com//rc/clk?jk=58a499ed20ba...
4,Chef de Cuisine - Eataly Flatiron,Full-time,Eataly North America,,Weekend availability+2,"New York, NY 10010 (Flatiron area)",Just posted,23-06-08,"Directs employee hiring, assignment, training,...",https://www.indeed.com//rc/clk?jk=0ff53943c907...
...,...,...,...,...,...,...,...,...,...,...
70,Server/Waiter/Waitress,"Full-time, Part-time",Miss Lily's - East Village,Estimated $38.3K - $48.5K a year,,"New York, NY 10009 (East Village area)",Today,23-06-08,At least 2 year experience of table service in...,https://www.indeed.com//rc/clk?jk=ed1e8088a1e9...
71,Host/Hostess,Full-time,Per Se,$16 - $18 an hour,,"New York, NY",Today,23-06-08,Company sponsored medical and dental plans wit...,https://www.indeed.com//rc/clk?jk=71d47b3a47bf...
72,Restaurant Host,"Full-time, Part-time",Estuary at ONE°15 Brooklyn Marina,,Monday to Friday+4,"Brooklyn, NY 11201",Today,23-06-08,Answering phone and assisting callers with res...,https://www.indeed.com//rc/clk?cmp=Estuary-at-...
73,Barback,Part-time,Miss Lily's - East Village,Estimated $35.4K - $44.8K a year,,"New York, NY 10009 (East Village area)",Today,23-06-08,We are seeking a person with 1-2 years experie...,https://www.indeed.com//rc/clk?jk=21dec338c758...


In [37]:
import numpy as np
for job in df['Apply_link']:
    #for link in links:
    driver.get(job)
    time.sleep(random.uniform(4, 5))
    soup = BeautifulSoup(driver.page_source, features="html.parser")
    #print(soup.text)
    all_metadata = soup.find_all("div", "css-tvvxwd ecydgvn1")
    rows =[i.get_text(strip=True) for i  in soup.find_all('div', 'css-tvvxwd ecydgvn1') if i.get_text(strip=True)!=""]
    job_type = []
    for i in rows:
        if "$" in i:
            salary = i
            rows.remove(i)
    for j in rows:
        if "Full-time" == j:
            job_type.append(j)
            rows.remove(j)
    for k in rows:
        if "Part-time" == k:
            job_type.append(k)
            rows.remove(k)
    
    schedule = ", ".join(rows)
    description = soup.find("div", "jobsearch-jobDescriptionText").text.replace("\n", " ").strip()
    print(salary)
    print(", ".join(job_type))
    print(schedule)
    print(description)
    df.loc[df['Apply_link'] ==  job, 'Salary'] = salary
    df.loc[df['Apply_link'] ==  job, 'Job_Type'] = ", ".join(job_type)
    df.loc[df['Apply_link'] ==  job, 'Schedule'] = schedule
    df.loc[df['Apply_link'] == job, 'Summary'] = description

    #np.where(final_df['score'] >3, 'positive', np.nan)


$17.50 an hour
Full-time
Monday to Friday
Restaurant Associates      We have an opening for a full-time DISHWASHER position.  Location: 111 8th Avenue 5th Floor, New York, NY 10011-5201 Note: online applications accepted only.  Schedule: Full-time schedule; Monday - Friday, hours may vary. More details upon interview!  Requirement: Prior dishwasher experience is required.  Starting Pay: $17.50 per hour    We Make Applying Easy! Want to apply to this job via text messaging? Text JOB to 75000 and search requisition ID number 1194723.  The advertised program is an AI recruiting assistant that helps you apply to jobs with Compass Group. Message frequency varies. Message and data rates may apply. Text STOP to opt out or HELP for help. Terms and conditions: http://olivia.paradox.ai/moSkg   This position is eligible for an Employee Referral Bonus! If you know someone that would be a great fit for this role, you can refer them to this position and potentially earn an Employee Referral Bonus! C

$22 an hour
Full-time
Weekend availability, Evenings as needed, Holidays
JOB SUMMARY    Serve food courses and alcoholic beverages To guests. Set tables according to type of event and service standards. Answer questions on menu selections. Communicate with the kitchen regarding menu questions, the length of wait, recook orders, and product availability. Communicate additional meal requirements, allergies, dietary needs, and special requests to the kitchen. Record transaction in MICROS system at time of order. Check in with guests to ensure satisfaction with each food course and/or beverages. Maintain cleanliness of work areas, china, glass, etc., throughout the day. Complete closing duties, including restocking items, turning off lights, etc. Present physical and accurate check to guest and process payment.    CORE WORK ACTIVITIES  Follow all company and safety and security policies and procedures; report accidents, injuries, and unsafe work conditions to manager; and complete safety t

$85,000 - $95,000 a year
Full-time, Part-time
Weekend availability, Holidays
Shift Supervisor   Our Hourly Shift Supervisors not only have an appetite for bottomless fun & compassion for our crazy loyal guests, they manage day to day operations of the restaurant during scheduled mid and closing shifts under the supervision of exempt management. Candidates and current Team Members who demonstrate our B.U.R.G.E.R values may be selected by exempt management for the Shift Supervisor position and will undergo on-the-job training for specific management tasks and leadership growth. As a part of the leadership team, they are an ambassador of Company & Brand Equity Standards to ensure the loyalty of our guests and profit maximization. They are master resolvers with guest interactions, driven to optimize profits, and ensure product quality and restaurant cleanliness. Candidates and current Team Members selected for this role may work in this position as well as other hourly roles they are certi

$80,000 - $90,000 a year
Part-time
8 hour shift, Weekend availability, Holidays
Performs all functions related to proper food preparation and maintenance of the cold case, fresh pack, salad, and hot bars. Ensures food quality and presentation and performs duties related to stocking and sanitation. All Whole Foods Market Retail jobs require ensuring a positive company image by providing courteous, friendly, and efficient service to customers and Team Members at all times. All positions must be performed in accordance with team and store Standard Operating Procedures. Further, Team Members must be prepared and able to perform the duties inherent in other Team Member job descriptions. All positions must strive to support WFM core values and goals, promote national, regional, and store programs and initiatives, and ensure adherence to all applicable health and safety regulations including Food Safety and regulatory duties required in the department.   Responsibilities   Prepares food items

$80,000 - $90,000 a year
Full-time, Part-time
Weekend availability, Evenings as needed, Holidays
Role Overview    Our Busser Team Members are responsible for efficiently maintaining a clean and sanitary dining room for our guests. They provide the dependable and fast service that Chili's is known for. If you take pride in great teamwork and communication, then we want to hear from you!    Fast hiring process  Flexible part-time or full-time schedule  Growth opportunities  Great team atmosphere and culture    Responsibilities  Maintain a clean dining room by clearing dishes, sweeping dining room floors, stocking supplies, sanitizing contact surfaces, and resetting tables for the next guests  Perform duties to support the service team as instructed by a manager  Work quickly to provide friendly service and keep up with the pace of the restaurant and team  Requires some shifts on evenings, weekends, and holidays    About Us    Chili's was born in Dallas, Texas in 1975. Since then, we've b

$60,000 - $65,000 a year


Descripción de Propiedad:  Embassy Suites by Hilton -Berkeley Heights, located in New Jersey, is seeking dedicated individuals to join our team! As a job applicant, you'll have the opportunity to work in a modern, all-suite hotel that offers exceptional service and amenities to our guests. With positions available in front desk, housekeeping, food and beverage, event planning, and more, there are ample opportunities for career growth and advancement. Our hotel features spacious suites, upscale facilities, and a warm and welcoming environment, creating an exciting work environment. As a member of the Embassy Suites Berkeley Heights team, you'll have the chance to provide outstanding service to our guests, work in a supportive team, and be a part of the renowned Hilton brand. Join us in delivering memorable hospitality experiences and become a valued member of our team at Embassy Suites Berkeley Heights!
$60,000 - $65,000 a year
Full-time

Role Overview    Our 

$15 an hour
Full-time, Part-time

In 2007, Jacopo Giustiniani opened the first Felice on Manhattan’s Upper East Side, creating a unique space for New Yorkers inspired by the spirit of the Italian countryside and the wines of his family’s vineyard. Set in an inviting and cozy atmosphere, Felice offers an extensive, Italian-driven wine list alongside a rustic menu of authentic Tuscan dishes. Today, there are several locations found across New York City, Brooklyn, Roslyn, and Florida.  You’ll be the face of our restaurant and responsible for our customers’ experiences. You need to be quick on your feet (literally and metaphorically) and have a polite and friendly attitude. If you enjoy talking to people and thrive in a fast-paced workplace, we’d like to meet you. Your ultimate goal will be to provide high-quality service that will help us maintain and attract customers.   Job Duties  Prepare restaurant tables with special attention to sanitation and order   Attend to customers upon entran

$15 an hour
Full-time
Weekend availability, Day shift
The Dinex Group, Chef Daniel Boulud’s internationally acclaimed restaurant group, seeks a dedicated and hospitable reservationist for the new multi-fine dining concept, Centurion® New York. The newest property is an exclusive, elegant space that will provide guests the highest level of service and hospitality. We are committed to creating a unique experience for each one of our guests and are looking for experienced and enthusiastic hospitality professionals to join our team.  The Dinex group is currently seeking Lounge Servers for this new venture.  The Lounge Server is responsible for beverage service of all tables and seating areas of the bar and lounges. The ideal candidate will have the ability to work under pressure, communicate effectively, multi-task and have strong attention to detail. This is a great learning opportunity for industry professionals with bartending experience.  Requirements include attention to detail, organ

$15 an hour
Full-time

We are seeking passionate, full-time Busser/Runners to join our opening team!  We offer great benefits:  PTO (paid time off) Health Care Coverage  Company covers 50% of the total premium cost!  Commuter benefits, flexible spending and dependent care accounts 401K  Matching up to 6%!   Ideal candidates possess:  Previous bussing/running experience in a high volume environment A passion for hospitality and food and beverage Working knowledge of POS systems Open availability   Please be advised, the hourly rate is $15.00 per hour less a $5.00 tip credit.   La Pecora Bianca is a proud Equal Opportunity Employer.  More detail about La Pecora Bianca - UWS part of La Pecora Bianca
$20 - $25 an hour
Full-time
10 hour shift
Le Rock is the newest restaurant from Frenchette chef owners Lee Hanson and Riad Nasr. Praised for their updated French fare, exciting, all natural wine program and warm hospitality, Le Rock has been critically acclaimed with 3 stars from the New York 

AttributeError: 'NoneType' object has no attribute 'text'

In [40]:
df

,Title,Job_Type,Company,Salary,Schedule,Location,Date_Posted,Current_Date,Summary,Apply_link
0,DISHWASHER (FULL-TIME),Full-time,Restaurant Associates,$17.50 an hour,Monday to Friday,"New York, NY 10011 (Chelsea area)",Just posted,23-06-08,Restaurant Associates We have an opening ...,https://www.indeed.com//rc/clk?jk=b86ec462493f...
1,GRILL COOK (FULL-TIME),Full-time,Restaurant Associates,$22 an hour,Monday to Friday,"New York, NY 10019 (Midtown area)",Just posted,23-06-08,Restaurant Associates We have an opening ...,https://www.indeed.com//rc/clk?jk=3fb1d1417712...
2,Server,Full-time,Marriott Vacations Worldwide,$22 an hour,"Weekend availability, Evenings as needed, Holi...","New York, NY 10018 (Garment District area)",Just posted,23-06-08,JOB SUMMARY Serve food courses and alcoholi...,https://www.indeed.com//rc/clk?jk=fa3d00337861...
3,Server,,Dos Caminos - Meatpacking,$22 an hour,,"New York, NY 10014 (West Village area)",Just posted,23-06-08,JOIN A WINNING TEAM! SERVER This isn’t ju...,https://www.indeed.com//rc/clk?jk=58a499ed20ba...
4,Chef de Cuisine - Eataly Flatiron,Full-time,Eataly North America,"$70,000 - $85,000 a year","Weekend availability, Evenings as needed, Holi...","New York, NY 10010 (Flatiron area)",Just posted,23-06-08,Company Description Eataly is the world’s la...,https://www.indeed.com//rc/clk?jk=0ff53943c907...
...,...,...,...,...,...,...,...,...,...,...
70,Server/Waiter/Waitress,Full-time,Miss Lily's - East Village,$16 - $20 an hour,Part-time,"New York, NY 10009 (East Village area)",Today,23-06-08,At least 2 year experience of table service in...,https://www.indeed.com//rc/clk?jk=ed1e8088a1e9...
71,Host/Hostess,Full-time,Per Se,$16 - $18 an hour,,"New York, NY",Today,23-06-08,Company sponsored medical and dental plans wit...,https://www.indeed.com//rc/clk?jk=71d47b3a47bf...
72,Restaurant Host,Full-time,Estuary at ONE°15 Brooklyn Marina,$12 - $20 an hour,"Part-time, 8 hour shift, Weekend availability,...","Brooklyn, NY 11201",Today,23-06-08,Answering phone and assisting callers with res...,https://www.indeed.com//rc/clk?cmp=Estuary-at-...
73,Barback,Part-time,Miss Lily's - East Village,$12 - $20 an hour,,"New York, NY 10009 (East Village area)",Today,23-06-08,We are seeking a person with 1-2 years experie...,https://www.indeed.com//rc/clk?jk=21dec338c758...


In [34]:
df.Apply_link[18]

'https://www.indeed.com//rc/clk?jk=fb9ad21bda517c8a&fccid=4bb0296fa44ba671&vjs=3'

In [33]:
data = df.to_dict(orient="records")
db.Job_Posts.insert_many(data)

In [14]:
df["Summary"][6]

"Beverage-wise, K'Far will offer a full coffee drink menu and cocktails with nods to Isreal.\nOpen schedule availability is preferred, and open minds are a must."

In [15]:
b

'Full-time'